In [29]:
import rasterio
import rasterio.features
import rasterio.warp
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot
import numpy as np
import pandas as pd
import os

In [30]:
df = pd.read_csv('data_processed.csv')

In [31]:
#x: longitud y: latitud
x = -72.6983	
y = -42.4087
df

,Región,Comuna,Tipo Remoción en masa,Factor desencadenante,Fecha Evento,Longitud,Latitud,Sistema Georeferencia,Cota (m.s.n.m)
0,Los Lagos,Castro,Flujo,Precipitaciones,07/03/2001,-73.7855,-42.4686,WGS 84- UTM 18 S,-
1,Los Lagos,Castro,Flujo,Precipitaciones,09/03/2001,-73.7855,-42.4686,WGS 84- UTM 18 S,-
2,Los Lagos,Chaitén,Deslizamiento,Precipitaciones,03/05/2002,-72.6983,-42.4087,WGS 84- UTM 18 S,-
3,Los Lagos,Río Negro,Deslizamiento,Precipitaciones,09/08/2002,-73.7889,-40.7083,WGS 84- UTM 18 S,6
4,Los Lagos,Osorno,Deslizamiento,Precipitaciones,22/08/2007,-73.6047,-40.5750,WGS 84- UTM 18 S,126
...,...,...,...,...,...,...,...,...,...
1242,Biobío,Tomé,Deslizamiento,Precipitaciones,11/07/2006,-72.9781,-36.5932,WGS 84/ UTM 18 S,40
1243,Biobío,Talcahuano,Caída,Precipitaciones,15/06/1991,-73.1032,-36.7543,WGS 84/ UTM 18 S,-
1244,Biobío,Talcahuano,Caída,Precipitaciones,16/06/1991,-73.1032,-36.7543,WGS 84/ UTM 18 S,-
1245,Biobío,Talcahuano,Caída,Precipitaciones,17/06/1991,-73.1032,-36.7543,WGS 84/ UTM 18 S,-


In [32]:
path

'C:\\Users\\ivo\\Desktop\\mlabbe\\script_geotiff_data_extraction\\ROSETTA_MEAN\\theta_r_15-30cm.tif'

In [41]:
path = r'ROSETTA_MEAN/theta_r_15-30cm.tif'

path_PIRange = r'PIRange-002'
path_rosetta_mean = r'ROSETTA_MEAN'
path_rosetta_sd = r'ROSETTA_SD'
path_SoilMaps_mean = r'SoilMaps_MEAN'
#path_SoilMaps_mean_2 = r'SoilMaps_MEAN'
path_Textural_Classes = r'Textural_Classes'
all_paths = [path_PIRange, path_rosetta_mean, path_rosetta_sd, path_SoilMaps_mean, path_SoilMaps_mean_2, path_Textural_Classes]
all_paths

['PIRange-002',
 'ROSETTA_MEAN',
 'ROSETTA_SD',
 'SoilMaps_MEAN',
 'C:\\Users\\ivo\\Desktop\\mlabbe\\script_geotiff_data_extraction\\SoilMaps_MEAN',
 'Textural_Classes']

In [53]:
def archivos_extraccion(paths):
    archivos = []
    for path in paths:
        for elemento in os.listdir(path):
            ruta_archivo = os.path.join(path, elemento)
            if os.path.isfile(ruta_archivo) and  ruta_archivo.lower().endswith('.tif'):
                archivos.append(elemento)
    return archivos

archivos = archivos_extraccion(all_paths)
archivos[0:3]

['PIRange_Bulkd.0-5cm.tif',
 'PIRange_Bulkd.100-200cm.tif',
 'PIRange_Bulkd.15-30cm.tif']

In [36]:
"""with rasterio.open(path) as dataset:
    mask = dataset.dataset_mask()
    for geom, val in rasterio.features.shapes(
        mask, transform=dataset.transform):
        geom = rasterio.warp.transform_geom(dataset.crs, 'EPSG:4326', geom, precision=6)
        
    print(geom)"""

"with rasterio.open(path) as dataset:\n    mask = dataset.dataset_mask()\n    for geom, val in rasterio.features.shapes(\n        mask, transform=dataset.transform):\n        geom = rasterio.warp.transform_geom(dataset.crs, 'EPSG:4326', geom, precision=6)\n        \n    print(geom)"

In [51]:
#funcions de lectura de tif
#x_,y_ = dataset_.index(-71.838400,-39.554800)
path = r'ROSETTA_MEAN/theta_r_15-30cm.tif'
dataset_ = rasterio.open(path)
x_,y_ = dataset_.index(x,y)
print(x,y)
print(x_,y_)
#x: longitud y: latitud
banda=dataset_.read(1)

print(banda[x_][y_])

-72.6983 -42.4087
25090 3297
0.087


In [54]:
def tif_extraccion(archivos):
    

SyntaxError: unexpected EOF while parsing (1888948733.py, line 1)

In [52]:
new_df = pd.concat([df.Longitud, df.Latitud], axis = 1)
new_df

,Longitud,Latitud
0,-73.7855,-42.4686
1,-73.7855,-42.4686
2,-72.6983,-42.4087
3,-73.7889,-40.7083
4,-73.6047,-40.5750
...,...,...
1242,-72.9781,-36.5932
1243,-73.1032,-36.7543
1244,-73.1032,-36.7543
1245,-73.1032,-36.7543
